<a href="https://colab.research.google.com/github/Alyona-Stankova/Alyona-Stankova/blob/main/Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка модулей

In [10]:
!pip install blocksnet ipykernel -q

In [11]:
from blocksnet import BlocksGenerator
from blocksnet import City

In [12]:
!pip install mapclassify -q

In [13]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
from shapely.geometry import mapping


# Жилые здания

In [14]:
area = "Московский район, Санкт-Петербург, Россия"
buildings = ox.geometries_from_place(area, tags={'building': True})

In [26]:
  # подгрузить файл с подготовленный жилыми зданиями с полями "парковочные_места_СИМ", "зарядные_места_СИМ", "количество_квартир"
buildings = gpd.read_file('/content/жилые_дома.geojson')

In [ ]:
buildings # проверка данных

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

ref  \
element_type osmid             
node         879400368   NaN   
             1336102521  NaN   
             1353829297  NaN   
             1404352817  NaN   
             1405716938  NaN   
...                      ...   
relation     18273041    NaN   
             18273042    NaN   
             18273043    NaN   
             18335409    NaN   
             18349522    NaN   

                                                                  geometry  \
element_type osmid                                                           
node         879400368                           POINT (30.32475 59.85352)   
             1336102521                          POINT (30.31506 59.85180)   
             1353829297                          POINT (30.32104 59.85052)   
             1404352817                          POINT (30.29805 59.85136)   
             1405716938                          POINT (30.32428 59.85362)   
...                                                                    ...   
relation     18273041    POLYGON ((30.35587 59.83300, 30.35594 59.83300...   
             18273042    POLYGON ((30.35460 59.83265, 30.35466 59.83266...   
             18273043    POLYGON ((30.35455 59.83278, 30.35451 59.83289...   
             18335409    POLYGON ((30.32965 59.83571, 30.32960 59.83572...   
             18349522    POLYGON ((30.35659 59.83456, 30.35673 59.83457...   

                        entrance access height man_made design:year  \
element_type osmid                                                    
node         879400368       NaN    NaN    NaN      NaN         NaN   
             1336102521      NaN    NaN    NaN      NaN         NaN   
             1353829297      NaN    NaN    NaN      NaN         NaN   
             1404352817      NaN    NaN    NaN      NaN         NaN   
             1405716938      NaN    NaN    NaN      NaN         NaN   
...                          ...    ...    ...      ...         ...   
relation     18273041        NaN    NaN    NaN      NaN         NaN   
             18273042        NaN    NaN    NaN      NaN         NaN   
             18273043        NaN    NaN    NaN      NaN         NaN   
             18335409        NaN    NaN    NaN      NaN         NaN   
             18349522        NaN    NaN    NaN      NaN         NaN   

                        drive_through operator was:amenity  ... intermittent  \
element_type osmid                                          ...                
node         879400368            NaN      NaN         NaN  ...          NaN   
             1336102521           NaN      NaN         NaN  ...          NaN   
             1353829297           NaN      NaN         NaN  ...          NaN   
             1404352817           NaN      NaN         NaN  ...          NaN   
             1405716938           NaN      NaN         NaN  ...          NaN   
...                               ...      ...         ...  ...          ...   
relation     18273041             NaN      NaN         NaN  ...          NaN   
             18273042             NaN      NaN         NaN  ...          NaN   
             18273043             NaN      NaN         NaN  ...          NaN   
             18335409             NaN      NaN         NaN  ...          NaN   
             18349522             NaN      NaN         NaN  ...          NaN   

                        natural addr:place area                      ways  \
element_type osmid                                                          
node         879400368      NaN        NaN  NaN                       NaN   
             1336102521     NaN        NaN  NaN                       NaN   
             1353829297     NaN        NaN  NaN                       NaN   
             1404352817     NaN        NaN  NaN                       NaN   
             1405716938     NaN        NaN  NaN                       NaN   
...                         ...        ...  ...                       ...   
relation    

In [ ]:
buildings.explore()

Метод

In [27]:
def calculate_lots(geos_layer):
    # Проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ', 'количество квартир']):

        # Проверяем минимальное необходимое количество мест
        required_parking = round(geos_layer['количество квартир'] * 0.8)
        required_charging = round(required_parking * 0.1)

        # Обновляем парковочные места только если текущее значение меньше требуемого
        if (geos_layer['парковочные_места_СИМ'] < required_parking).any():
            geos_layer.loc[geos_layer['парковочные_места_СИМ'] < required_parking, 'парковочные_места_СИМ'] = required_parking
        else:
            print("Достаточное количество парковочных мест")

        # Обновляем зарядные места только если текущее значение меньше требуемого
        if (geos_layer['зарядные_места_СИМ'] < required_charging).any():
            geos_layer.loc[geos_layer['зарядные_места_СИМ'] < required_charging, 'зарядные_места_СИМ'] = required_charging
        else:
            print("Достаточное количество зарядных мест")

    else:
        # Обработка геослоя
        geos_layer = geos_layer[geos_layer.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()  # определяем локальную систему координат
        geos_layer = geos_layer.to_crs(local_crs)  # переводим здания в локальную систему координат

        # Преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
        geos_layer['building:levels'] = pd.to_numeric(geos_layer['building:levels'], errors='coerce')
        # Заполним пропущенные значения (NaN) нулями
        geos_layer = geos_layer.fillna(0)

        # Добавляем или преобразуем необходимые атрибуты
        # 1. Количество этажей (number_of_floors)
        geos_layer['number_of_floors'] = geos_layer.apply(
            lambda x: max(1, x['building:levels']), axis=1
        )

        # 2. Площадь застройки (footprint_area) - как площадь геометрии (основание здания)
        geos_layer['footprint_area'] = geos_layer.geometry.area

        # 3. Общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
        geos_layer['build_floor_area'] = geos_layer['footprint_area'] * geos_layer['number_of_floors']

        # 4. Жилая площадь (living_area)
        residential_tags = ['residential', 'house', 'apartments', 'detached', 'terrace', 'dormitory']
        geos_layer['living_area'] = geos_layer.apply(
            lambda x: 0.8 * x['build_floor_area'] if x.get('building') in residential_tags else 0,
            axis=1
        )

        # Нежилая площадь будет 20% от общей площади этажей
        geos_layer['non_living_area'] = geos_layer['build_floor_area'] - geos_layer['living_area']

        # 5. Население (population) жилых зданий
        geos_layer['population'] = geos_layer.apply(
            lambda x: 48 * x['number_of_floors'] if x.get('building') in residential_tags else 0,
            axis=1
        )

        # Удаляем здания без жилой площади
        geos_layer['living_area'] = pd.to_numeric(geos_layer['living_area'], errors='coerce')
        geos_layer = geos_layer.loc[geos_layer['living_area'] != 0]

        # Расчёт количества квартир
        geos_layer['количество квартир'] = geos_layer['population'] / 4

        # Сохраняем только необходимые столбцы
        geos_layer = geos_layer[['addr:street', 'addr:housenumber', 'geometry', 'build_floor_area',
                                  'living_area', 'footprint_area', 'number_of_floors',
                                  'population', 'количество квартир']]

        # Расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = round(geos_layer['количество квартир'] * 0.8)
        geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)

        # Возвращаем к оригинальной системе координат
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer

In [28]:
buildings = calculate_lots(buildings)
buildings

Достаточное количество парковочных мест


,fid,id,addr:housenumber,addr:street,Square,Residens,количество квартир,парковочные_места_СИМ,зарядные_места_СИМ,geometry
0,1,relation/5651560,183-185,Московский проспект,5 000,130,32.50,26,3,"MULTIPOLYGON (((30.31917 59.85637, 30.31918 59..."
1,2,relation/5651561,183-185 литБ,Московский проспект,5000,130,32.50,26,3,"MULTIPOLYGON (((30.31491 59.85675, 30.31487 59..."
2,3,relation/15969900,51 к1,Варшавская улица,"4 770,40",173,43.25,35,4,"POLYGON ((30.31349 59.85597, 30.31344 59.85597..."
3,4,relation/16920612,101,Новоизмайловский проспект,"7 846,86",289,72.25,58,6,"POLYGON ((30.30309 59.85501, 30.30287 59.85501..."
4,5,relation/16920615,85,Новоизмайловский проспект,"3 022,49",118,29.50,24,2,"POLYGON ((30.30308 59.85514, 30.30308 59.85510..."
...,...,...,...,...,...,...,...,...,...,...
213,214,way/50905618,76 к2,Краснопутиловская улица,"5 985,00",550,137.50,110,11,"POLYGON ((30.30111 59.85490, 30.30123 59.85492..."
214,215,way/140538236,6Б,улица Фрунзе,"2 680,00",106,26.50,21,2,"POLYGON ((30.31851 59.86247, 30.31851 59.86255..."
215,216,way/140538239,6А,улица Фрунзе,"9 712,00",383,95.75,77,8,"POLYGON ((30.31732 59.86245, 30.31757 59.86245..."
216,217,way/253313631,173,Московский проспект,"8 081,50",227,56.75,45,5,"POLYGON ((30.31949 59.86148, 30.31932 59.86148..."


Сохранение файла

In [ ]:
buildings.to_file('жилые_здания_СИМ.geojson')

Визуализация

In [29]:
buildings['centroid'] = buildings.geometry.centroid

# Добавление столбцов для широты и долготы центроидов
buildings['latitude'] = buildings['centroid'].y
buildings['longitude'] = buildings['centroid'].x

# Удаление временного столбца 'centroid', если он не нужен
buildings = buildings.drop(columns='centroid')
buildings

<ipython-input-29-063c58d45c45>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings['centroid'] = buildings.geometry.centroid


,fid,id,addr:housenumber,addr:street,Square,Residens,количество квартир,парковочные_места_СИМ,зарядные_места_СИМ,geometry,latitude,longitude
0,1,relation/5651560,183-185,Московский проспект,5 000,130,32.50,26,3,"MULTIPOLYGON (((30.31917 59.85637, 30.31918 59...",59.856029,30.317758
1,2,relation/5651561,183-185 литБ,Московский проспект,5000,130,32.50,26,3,"MULTIPOLYGON (((30.31491 59.85675, 30.31487 59...",59.856883,30.315674
2,3,relation/15969900,51 к1,Варшавская улица,"4 770,40",173,43.25,35,4,"POLYGON ((30.31349 59.85597, 30.31344 59.85597...",59.855198,30.313656
3,4,relation/16920612,101,Новоизмайловский проспект,"7 846,86",289,72.25,58,6,"POLYGON ((30.30309 59.85501, 30.30287 59.85501...",59.854620,30.303592
4,5,relation/16920615,85,Новоизмайловский проспект,"3 022,49",118,29.50,24,2,"POLYGON ((30.30308 59.85514, 30.30308 59.85510...",59.855260,30.303052
...,...,...,...,...,...,...,...,...,...,...,...,...
213,214,way/50905618,76 к2,Краснопутиловская улица,"5 985,00",550,137.50,110,11,"POLYGON ((30.30111 59.85490, 30.30123 59.85492...",59.854882,30.300929
214,215,way/140538236,6Б,улица Фрунзе,"2 680,00",106,26.50,21,2,"POLYGON ((30.31851 59.86247, 30.31851 59.86255...",59.862709,30.318380
215,216,way/140538239,6А,улица Фрунзе,"9 712,00",383,95.75,77,8,"POLYGON ((30.31732 59.86245, 30.31757 59.86245...",59.862689,30.317431
216,217,way/253313631,173,Московский проспект,"8 081,50",227,56.75,45,5,"POLYGON ((30.31949 59.86148, 30.31932 59.86148...",59.860971,30.318963


In [30]:
# Создание DataFrame из buildings
df = pd.DataFrame(buildings)

# Используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')
buildings_group = folium.FeatureGroup(name='Жилые здания')

folium.TileLayer('CartoDB positron').add_to(m)

# Итерация по строкам DataFrame
for idx, row in df.iterrows():
    # Проверяем наличие необходимых колонок
    if 'парковочные_места_СИМ' in row and 'зарядные_места_СИМ' in row:
        # Размер точки на основе количества парковочных мест
        parking_size = row['парковочные_места_СИМ'] / 9  # Измените делитель по необходимости
        charging_size = row['зарядные_места_СИМ'] / 7  # Измените делитель по необходимости

        # Добавляем круговые маркеры для парковочных мест
        parking_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=parking_size,
                color=None,
                fill=True,
                fill_color='#BFA181',
                fill_opacity=0.6,
                tooltip=f"Парковочные места: {row['парковочные_места_СИМ']}"
            )
        )

        # Добавляем круговые маркеры для зарядных мест
        charging_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=charging_size,
                color=None,
                fill=True,
                fill_color='#0A1828',
                fill_opacity=0.9,
                tooltip=f"Зарядные места: {row['зарядные_места_СИМ']}"  # Исправлено на нужное поле
            )
        )

# Итерация для добавления зданий на карту
for index, row in df.iterrows():
    # Проверяем наличие геометрии
    if 'geometry' in row and row['geometry'] is not None:
        geometry = row['geometry']

        # Добавляем полигональный слой зданий на карту
        buildings_group.add_child(
            folium.GeoJson(
                geometry,  # Используем всю геометрию
                color=None,
                fill=True,
                fill_color='#96C2DB',
                fill_opacity=0.5,
                tooltip=f"Адрес: {row['addr:street']} {row['addr:housenumber']}"
            )
        )

# Добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)
m.add_child(buildings_group)

# Добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# Отображаем карту
m

Сохранение карты

In [19]:
m.save('map_buildings.html')

# Сервисы по площади




In [20]:
city_name = "Московский район, Санкт-Петербург, Россия"

# Загружаем школы (amenity=school)
tags = {'building': 'office'}
service_1 = ox.geometries_from_place(city_name, tags)

<ipython-input-20-2661cf778c7c>:5: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  service_1 = ox.geometries_from_place(city_name, tags)


In [ ]:
# подгрузить файл с подготовленный жилыми зданиями с полями "парковочные_места_СИМ", "зарядные_места_СИМ", "Площадь"
service_1 = gpd.read_file('/content/БЦ_3.geojson')


In [ ]:
service_1 # проверка данных

,fid,Площадь,Этажи,Вместимость,Велосипеды,Электросамокаты,Электровелосипеды,Другое,парковочные_места_СИМ,зарядные_места_СИМ,geometry
0,1,21261,None,None,None,None,8,None,100,2,"POLYGON ((30.30073 59.85300, 30.30072 59.85354..."
1,2,12875,None,None,None,None,4,None,50,1,"POLYGON ((30.30241 59.85410, 30.30229 59.85409..."
2,3,2367,None,None,None,None,1,None,5,1,"POLYGON ((30.30633 59.85449, 30.30660 59.85449..."


In [ ]:
service_1.explore()

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


Метод

In [21]:
def calculate_lots_service_1(geos_layer):
    # Проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ', 'Площадь']):

        # Проверяем минимальное необходимое количество мест
        required_parking = round(geos_layer['Площадь'] * 0.004)
        required_charging = round(required_parking * 0.1)

        # Обновляем парковочные места только если текущее значение меньше требуемого
        if (geos_layer['парковочные_места_СИМ'] < required_parking).any():
            geos_layer.loc[geos_layer['парковочные_места_СИМ'] < required_parking, 'парковочные_места_СИМ'] = required_parking
        else:
            print("Достаточное количество парковочных мест")

        # Обновляем зарядные места только если текущее значение меньше требуемого
        if (geos_layer['зарядные_места_СИМ'] < required_charging).any():
            geos_layer.loc[geos_layer['зарядные_места_СИМ'] < required_charging, 'зарядные_места_СИМ'] = required_charging
        else:
            print("Достаточное количество зарядных мест")

    else:
        # Обработка геослоя
        geos_layer = geos_layer[geos_layer.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()  # определяем локальную систему координат
        geos_layer = geos_layer.to_crs(local_crs)  # переводим здания в локальную систему координат

        # Преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
        geos_layer['building:levels'] = pd.to_numeric(geos_layer['building:levels'], errors='coerce')
        # Заполним пропущенные значения (NaN) нулями
        geos_layer = geos_layer.fillna(0)

        # Добавляем или преобразуем необходимые атрибуты
        # 1. Количество этажей (number_of_floors)
        geos_layer['number_of_floors'] = geos_layer.apply(
             lambda x: x['building:levels'] if x['building:levels'] > 1 else 1,
             axis=1
        )

        # 2. лощадь застройки (footprint_area) - как площадь геометрии (основание здания)
        geos_layer['footprint_area'] = geos_layer.geometry.area

        # 3. Общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
        geos_layer['Площадь_2'] = geos_layer['footprint_area'] * geos_layer['number_of_floors']
        geos_layer['Площадь'] = geos_layer['Площадь_2'] * 0.4


        # Теперь удалим все остальные столбцы, кроме 'geometry', 'build_floor_area', 'living_area',
        # 'footprint_area', 'number_of_floors', 'population', 'addr:housenumber', 'addr:street'
        geos_layer = geos_layer[['geometry', 'Площадь', 'footprint_area', 'number_of_floors', 'addr:housenumber', 'addr:street']]

        # Расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = round(geos_layer['Площадь'] * 0.004)
        geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)

        # Возвращаем к оригинальной системе координат
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer

In [22]:
service_1 = calculate_lots_service_1(service_1)
service_1

,geometry,Площадь,footprint_area,number_of_floors,addr:housenumber,addr:street,парковочные_места_СИМ,зарядные_места_СИМ
0,"POLYGON ((30.31021 59.86434, 30.30989 59.86434...",11205.908869,2154.982475,13.0,21,Бассейная улица,45.0,4.0
1,"POLYGON ((30.32803 59.84324, 30.32803 59.84312...",291.624818,364.531023,2.0,5,улица Орджоникидзе,1.0,0.0
2,"POLYGON ((30.33523 59.84342, 30.33554 59.84342...",99.183379,247.958447,1.0,63,проспект Юрия Гагарина,0.0,0.0
3,"POLYGON ((30.33518 59.84543, 30.33492 59.84542...",516.752353,645.940441,2.0,55,проспект Юрия Гагарина,2.0,0.0
4,"POLYGON ((30.33291 59.84871, 30.33291 59.84883...",283.078924,353.848654,2.0,37,Алтайская улица,1.0,0.0
...,...,...,...,...,...,...,...,...
125,"POLYGON ((30.31466 59.80896, 30.31453 59.80900...",895.158155,2237.895387,1.0,7,Стартовая улица,4.0,0.0
126,"POLYGON ((30.30556 59.80619, 30.30599 59.80638...",6767.753026,8459.691282,2.0,17 с2,Стартовая улица,27.0,3.0
127,"POLYGON ((30.33565 59.87583, 30.33569 59.87586...",15278.257826,6365.940761,6.0,1,проспект Юрия Гагарина,61.0,6.0
128,"POLYGON ((30.32141 59.89481, 30.32171 59.89480...",25852.352405,8078.860127,8.0,8 литК,Черниговская улица,103.0,10.0


Сохренение файла

In [ ]:
service_1.to_file('сервис_по_площади_СИМ_1.geojson')

Визуализация

In [23]:
service_1['centroid'] = service_1.geometry.centroid

# Добавление столбцов для широты и долготы центроидов
service_1['latitude'] = service_1['centroid'].y
service_1['longitude'] = service_1['centroid'].x

# Удаление временного столбца 'centroid', если он не нужен
service_1 = service_1.drop(columns='centroid')

service_1

<ipython-input-23-7349c7516dfc>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  service_1['centroid'] = service_1.geometry.centroid


,geometry,Площадь,footprint_area,number_of_floors,addr:housenumber,addr:street,парковочные_места_СИМ,зарядные_места_СИМ,latitude,longitude
0,"POLYGON ((30.31021 59.86434, 30.30989 59.86434...",11205.908869,2154.982475,13.0,21,Бассейная улица,45.0,4.0,59.864269,30.309489
1,"POLYGON ((30.32803 59.84324, 30.32803 59.84312...",291.624818,364.531023,2.0,5,улица Орджоникидзе,1.0,0.0,59.843176,30.327790
2,"POLYGON ((30.33523 59.84342, 30.33554 59.84342...",99.183379,247.958447,1.0,63,проспект Юрия Гагарина,0.0,0.0,59.843356,30.335388
3,"POLYGON ((30.33518 59.84543, 30.33492 59.84542...",516.752353,645.940441,2.0,55,проспект Юрия Гагарина,2.0,0.0,59.845627,30.335034
4,"POLYGON ((30.33291 59.84871, 30.33291 59.84883...",283.078924,353.848654,2.0,37,Алтайская улица,1.0,0.0,59.848772,30.333154
...,...,...,...,...,...,...,...,...,...,...
125,"POLYGON ((30.31466 59.80896, 30.31453 59.80900...",895.158155,2237.895387,1.0,7,Стартовая улица,4.0,0.0,59.809367,30.315071
126,"POLYGON ((30.30556 59.80619, 30.30599 59.80638...",6767.753026,8459.691282,2.0,17 с2,Стартовая улица,27.0,3.0,59.806220,30.306754
127,"POLYGON ((30.33565 59.87583, 30.33569 59.87586...",15278.257826,6365.940761,6.0,1,проспект Юрия Гагарина,61.0,6.0,59.874825,30.335724
128,"POLYGON ((30.32141 59.89481, 30.32171 59.89480...",25852.352405,8078.860127,8.0,8 литК,Черниговская улица,103.0,10.0,59.894466,30.322027


In [24]:
# Создание DataFrame из service_1
df = pd.DataFrame(service_1)

# Используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')
buildings_group = folium.FeatureGroup(name='Сервис_1')

folium.TileLayer('CartoDB positron').add_to(m)

# Итерация по строкам DataFrame
for idx, row in df.iterrows():
    # Проверяем наличие необходимых колонок
    if 'парковочные_места_СИМ' in row and 'зарядные_места_СИМ' in row:
        # Размер точки на основе количества парковочных мест
        parking_size = row['парковочные_места_СИМ'] / 10  # Измените делитель по необходимости
        charging_places = row['зарядные_места_СИМ'] if row['зарядные_места_СИМ'] > 0 else 1

        # Проверяем количество зарядных мест
        if charging_places >= 1:  # Условие для зарядных мест
            charging_size = charging_places / 10  # Измените делитель по необходимости

        # Добавляем круговые маркеры для парковочных мест
        parking_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=parking_size,
                color=None,
                fill=True,
                fill_color='#BFA181',
                fill_opacity=0.6,
                tooltip=f"Парковочные места: {row['парковочные_места_СИМ']}"
            )
        )

        # Добавляем круговые маркеры для зарядных мест
        charging_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=charging_size,
                color=None,
                fill=True,
                fill_color='#0A1828',
                fill_opacity=0.9,
                tooltip=f"Зарядные места: {row['зарядные_места_СИМ']}"  # Исправлено на нужное поле
            )
        )

# Итерация для добавления зданий на карту
for index, row in df.iterrows():
    # Проверяем наличие геометрии
    if 'geometry' in row and row['geometry'] is not None:
        geometry = row['geometry']

        # Добавляем полигональный слой зданий на карту
        buildings_group.add_child(
            folium.GeoJson(
                geometry,  # Используем всю геометрию
                color=None,
                fill=True,
                fill_color='#96C2DB',
                fill_opacity=0.5,
                tooltip=f"Адрес: {row['addr:street']} {row['addr:housenumber']}"
            )
        )

# Добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)
m.add_child(buildings_group)

# Добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# Отображаем карту
m

Сохранение карты

In [ ]:
m.save('map_service_1.html')

# Сервисы по вместимости

In [ ]:
 #Определяем территори.
city_name = "Московский район, Санкт-Петербург, Россия"

# Загружаем школы (amenity=school)
tags = {'building': 'school'}
service1 = ox.geometries_from_place(city_name, tags)

<ipython-input-161-c7eb4b048a52>:6: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  service1 = ox.geometries_from_place(city_name, tags)


In [31]:
  # подгрузить файл с подготовленный жилыми зданиями с полями "парковочные_места_СИМ", "зарядные_места_СИМ", "Вместимость"
service1 = gpd.read_file('/content/Школы_2.geojson')

In [ ]:
service1

,id,name,Велосипеды,Самокаты,Вместимость,расчёт,нужное количество сим,сумма,разница,электро,парковочные_места_СИМ,зарядные_места_СИМ,geometry
0,0.42,Школа № 510,1,2,510,0.041,10.20,3,7.20,1.020,5,1,"POLYGON ((30.29451 59.85466, 30.29451 59.85399..."
1,1.26,Школа № 684 «Берегиня»,3,6,742,0.085,14.84,9,5.84,1.484,15,0,"POLYGON ((30.29737 59.85960, 30.29732 59.85923..."
2,0.8200000000000001,Школа № 507,1,4,729,0.056,14.58,5,9.58,1.458,15,2,"POLYGON ((30.31614 59.86350, 30.31619 59.86301..."
3,1.8800000000000001,Школа № 495,4,9,641,0.147,12.82,13,-0.18,1.282,8,1,"POLYGON ((30.31005 59.86198, 30.31001 59.86121..."
4,1.24,Школа № 496,2,6,663,0.094,13.26,8,5.26,1.326,13,1,"POLYGON ((30.31041 59.85518, 30.31046 59.85444..."
5,0.84,Школа № 594,2,4,662,0.063,13.24,6,7.24,1.324,13,10,"POLYGON ((30.31708 59.86054, 30.31657 59.86074..."


In [ ]:
service1.explor()

Метод

In [32]:
def calculate_lots_service1(geos_layer):
    # Проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ', 'Вместимость']):

        # Проверяем минимальное необходимое количество мест
        required_parking = round(geos_layer['Вместимость'] * 0.02)
        required_charging = round(required_parking * 0.1)

        # Обновляем парковочные места только если текущее значение меньше требуемого
        if (geos_layer['парковочные_места_СИМ'] < required_parking).any():
            geos_layer.loc[geos_layer['парковочные_места_СИМ'] < required_parking, 'парковочные_места_СИМ'] = required_parking
        else:
            print("Достаточное количество парковочных мест")

        # Обновляем зарядные места только если текущее значение меньше требуемого
        if (geos_layer['зарядные_места_СИМ'] < required_charging).any():
            geos_layer.loc[geos_layer['зарядные_места_СИМ'] < required_charging, 'зарядные_места_СИМ'] = required_charging
        else:
            print("Достаточное количество зарядных мест")

    else:
    # Обработка геослоя
       geos_layer = geos_layer[geos_layer.geometry.type.isin(['Polygon'])]
       geos_layer = geos_layer.reset_index(drop=True)
       original_crs = geos_layer.crs
       local_crs = geos_layer.estimate_utm_crs()
       geos_layer = geos_layer.to_crs(local_crs) # переводим школы в локальную систему координат
       geos_layer.crs  # переводим здания в локальную систему координат

# Преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
       geos_layer['building:levels'] = pd.to_numeric(geos_layer['building:levels'], errors='coerce')

# Заполним пропущенные значения (NaN) нулями
       geos_layer = geos_layer.fillna(0)

# Добавляем или преобразуем необходимые атрибуты

# 1. Количество этажей (number_of_floors)
       geos_layer['number_of_floors'] = geos_layer.apply(
             lambda x: x['building:levels'] if x['building:levels'] > 1 else 1,
             axis=1
       )

# 2. Площадь застройки (footprint_area) - как площадь геометрии (основание здания)
       geos_layer['footprint_area'] = geos_layer.geometry.area

# 3. Общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
       geos_layer['Площадь'] = geos_layer['footprint_area'] * geos_layer['number_of_floors']
       geos_layer['Расчетная_площадь'] = geos_layer['Площадь'] - (geos_layer['Площадь'] * 0.6)
       geos_layer['Вместимость'] = geos_layer['Расчетная_площадь'] / 2.5

# Теперь удалим все остальные столбцы, кроме 'geometry', 'build_floor_area', 'living_area',
# 'footprint_area', 'number_of_floors', 'population'
       geos_layer = geos_layer[['geometry', 'Площадь', 'footprint_area', 'number_of_floors', 'addr:housenumber', 'addr:street', 'Расчетная_площадь', 'Вместимость']]

    # Расчёт парковочных и зарядных мест
       geos_layer['парковочные_места_СИМ'] = round(geos_layer['Вместимость'] * 0.02)
       geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)
       geos_layer = geos_layer.to_crs(original_crs)



    return geos_layer

In [33]:
service1 = calculate_lots_service1(service1)
service1

,id,name,Велосипеды,Самокаты,Вместимость,расчёт,нужное количество сим,сумма,разница,электро,парковочные_места_СИМ,зарядные_места_СИМ,geometry
0,0.42,Школа № 510,1,2,510,0.041,10.20,3,7.20,1.020,10,1,"POLYGON ((30.29451 59.85466, 30.29451 59.85399..."
1,1.26,Школа № 684 «Берегиня»,3,6,742,0.085,14.84,9,5.84,1.484,15,2,"POLYGON ((30.29737 59.85960, 30.29732 59.85923..."
2,0.8200000000000001,Школа № 507,1,4,729,0.056,14.58,5,9.58,1.458,15,2,"POLYGON ((30.31614 59.86350, 30.31619 59.86301..."
3,1.8800000000000001,Школа № 495,4,9,641,0.147,12.82,13,-0.18,1.282,13,1,"POLYGON ((30.31005 59.86198, 30.31001 59.86121..."
4,1.24,Школа № 496,2,6,663,0.094,13.26,8,5.26,1.326,13,1,"POLYGON ((30.31041 59.85518, 30.31046 59.85444..."
5,0.84,Школа № 594,2,4,662,0.063,13.24,6,7.24,1.324,13,10,"POLYGON ((30.31708 59.86054, 30.31657 59.86074..."


Сохранение файла

In [ ]:
service1.to_file('сервис_по_вместимости_СИМ1.geojson')

Визуализация

In [34]:
service1['centroid'] = service1.geometry.centroid

# Добавление столбцов для широты и долготы центроидов
service1['latitude'] = service1['centroid'].y
service1['longitude'] = service1['centroid'].x

# Удаление временного столбца 'centroid', если он не нужен
service1 = service1.drop(columns='centroid')

<ipython-input-34-beaa173df8b3>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  service1['centroid'] = service1.geometry.centroid


In [40]:
# Создание DataFrame из service1
df = pd.DataFrame(service1)

# Используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')
buildings_group = folium.FeatureGroup(name='Сервис1')

folium.TileLayer('CartoDB positron').add_to(m)

# Итерация по строкам DataFrame
for idx, row in df.iterrows():
    # Проверяем наличие необходимых колонок
    if 'парковочные_места_СИМ' in row and 'зарядные_места_СИМ' in row:
        # Размер точки на основе количества парковочных мест
        parking_size = row['парковочные_места_СИМ'] / 5  # Измените делитель по необходимости
        charging_places = row['зарядные_места_СИМ'] if row['зарядные_места_СИМ'] > 0 else 1

        # Проверяем количество зарядных мест
        if charging_places >= 1:  # Условие для зарядных мест
            charging_size = charging_places / 5  # Измените делитель по необходимости

        # Добавляем круговые маркеры для парковочных мест
        parking_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=parking_size,
                color=None,
                fill=True,
                fill_color='#BFA181',
                fill_opacity=0.6,
                tooltip=f"Парковочные места: {row['парковочные_места_СИМ']}"
            )
        )

        # Добавляем круговые маркеры для зарядных мест
        charging_group.add_child(
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),  # Используем координаты текущей строки
                radius=charging_size,
                color=None,
                fill=True,
                fill_color='#0A1828',
                fill_opacity=0.9,
                tooltip=f"Зарядные места: {row['зарядные_места_СИМ']}"  # Исправлено на нужное поле
            )
        )

# Итерация для добавления зданий на карту
for index, row in df.iterrows():
    # Проверяем наличие геометрии
    if 'geometry' in row and row['geometry'] is not None:
        geometry = row['geometry']

        # Добавляем полигональный слой зданий на карту
        buildings_group.add_child(
            folium.GeoJson(
                geometry,  # Используем всю геометрию
                color=None,
                fill=True,
                fill_color='#96C2DB',
                fill_opacity=0.5,
                tooltip=f"Сервис1:{row['name']}"
            )
        )

# Добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)
m.add_child(buildings_group)

# Добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# Отображаем карту
m

In [ ]:
m.save('map_service1.html')

# Остановки наземного танспорта

In [ ]:
#Определяем территори.
city_name_3 = "Московский район, Санкт-Петербург, Россия"

# Загружаем остановки
tags = {'highway': 'bus_stop'}
stops = ox.geometries_from_place(city_name_3, tags)


<ipython-input-37-1b5507f67fef>:6: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  stops = ox.geometries_from_place(city_name_3, tags)


In [ ]:
stops = gpd.read_file('/content/....geojson')

DriverError: Failed to open dataset (flags=68): /content/....geojson

In [ ]:
stops #проверяем данные

bench  bin  bus covered   highway  lit  \
element_type osmid                                                
node         352785098     yes  yes  yes     yes  bus_stop  yes   
             352785099     yes  yes  yes     yes  bus_stop  yes   
             410512560     yes  yes  yes     NaN  bus_stop  NaN   
             410525401     yes  yes  yes     NaN  bus_stop  yes   
             410525405     yes  yes  yes     NaN  bus_stop  yes   
...                        ...  ...  ...     ...       ...  ...   
             12386951663   yes  yes  yes     NaN  bus_stop  yes   
             12393618473   NaN  NaN  yes     NaN  bus_stop  NaN   
             12431558627   NaN  NaN  yes     NaN  bus_stop  NaN   
             12431558628   NaN  NaN  yes     NaN  bus_stop  NaN   
             12451658552   NaN  NaN  yes     NaN  bus_stop  NaN   

                                                 name public_transport  \
element_type osmid                                                       
node         352785098    станция метро «Парк Победы»         platform   
             352785099    станция метро «Парк Победы»         platform   
             410512560                 Южное кладбище         platform   
             410525401        Пулковская обсерватория         platform   
             410525405         торговый центр «Метро»         platform   
...                                               ...              ...   
             12386951663               Южное кладбище         platform   
             12393618473           Дунайский проспект         platform   
             12431558627                          NaN         platform   
             12431558628                          NaN         platform   
             12451658552              Бассейная улица         platform   

                         shelter tactile_paving  ... alt_name name:en  \
element_type osmid                               ...                    
node         352785098       yes             no  ...      NaN     NaN   
             352785099       yes             no  ...      NaN     NaN   
             410512560       yes             no  ...      NaN     NaN   
             410525401       yes             no  ...      NaN     NaN   
             410525405       yes             no  ...      NaN     NaN   
...                          ...            ...  ...      ...     ...   
             12386951663     yes             no  ...      NaN     NaN   
             12393618473     NaN            NaN  ...      NaN     NaN   
             12431558627     NaN            NaN  ...      NaN     NaN   
             12431558628     NaN            NaN  ...      NaN     NaN   
             12451658552     NaN            NaN  ...      NaN     NaN   

                         description check_date:bench name:ru  \
element_type osmid                                              
node         352785098           NaN              NaN     NaN   
             352785099           NaN              NaN     NaN   
             410512560           NaN              NaN     NaN   
             410525401           NaN              NaN     NaN   
             410525405           NaN              NaN     NaN   
...                              ...              ...     ...   
             12386951663         NaN              NaN     NaN   
             12393618473         NaN              NaN     NaN   
             12431558627         NaN              NaN     NaN   
             12431558628         NaN              NaN     NaN   
             12451658552         NaN              NaN     NaN   

                         departures_board name:de tram  ref pole  
element_type osmid                                                
node         352785098                NaN     NaN  NaN  NaN  NaN  
             352785099                NaN     NaN  NaN  NaN  NaN  
             410512560                NaN     NaN  NaN  NaN  NaN  
             410525401                NaN     NaN  NaN  NaN

In [ ]:
stops.explore()

Метод

In [ ]:
def calculate_stops(geos_layer):
    # Проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ']):

        # Заполняем NaN значения нулями, чтобы избежать ошибок
        geos_layer['парковочные_места_СИМ'] = geos_layer['парковочные_места_СИМ'].fillna(0)
        geos_layer['зарядные_места_СИМ'] = geos_layer['зарядные_места_СИМ'].fillna(0)

        # Проверяем минимальное необходимое количество зарядных мест
        if (geos_layer['парковочные_места_СИМ'] >= 5).all():
            # Если количество парковочных мест достаточно, ничего не делаем
            print("Достаточное количество парковочных мест для СИМ")
        else:
            geos_layer['парковочные_места_СИМ'] = 5

        if (geos_layer['зарядные_места_СИМ'] >= 1).all():
            # Если количество зарядных мест достаточно, ничего не делаем
            print("Достаточное количество зарядных мест для СИМ")
        else:
            geos_layer['зарядные_места_СИМ'] = 1

    else:
        # Обработка геослоя
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()
        geos_layer = geos_layer.to_crs(local_crs)  # переводим в локальную систему координат

        # Заполняем NaN значения нулями
        geos_layer = geos_layer.fillna(0)

        # Расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = 5
        geos_layer['зарядные_места_СИМ'] = 1
        # Теперь удалим все остальные столбцы, кроме 'geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ'
        geos_layer = geos_layer[['geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ']]
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer

In [ ]:
stops = calculate_stops(stops)
stops

Достаточное количество парковочных мест для СИМ
Достаточное количество зарядных мест для СИМ


,geometry,name,парковочные_места_СИМ,зарядные_места_СИМ,latitude,longitude
0,POINT (30.32310 59.86495),станция метро «Парк Победы»,5,1,59.864950,30.323100
1,POINT (30.32186 59.86478),станция метро «Парк Победы»,5,1,59.864780,30.321856
2,POINT (30.26419 59.76122),Южное кладбище,5,1,59.761223,30.264186
3,POINT (30.33221 59.77065),Пулковская обсерватория,5,1,59.770654,30.332215
4,POINT (30.32338 59.82484),торговый центр «Метро»,5,1,59.824835,30.323384
...,...,...,...,...,...,...
369,POINT (30.26351 59.76090),Южное кладбище,5,1,59.760897,30.263513
370,POINT (30.31448 59.83123),Дунайский проспект,5,1,59.831234,30.314478
371,POINT (30.32358 59.82056),0,5,1,59.820559,30.323583
372,POINT (30.32247 59.82034),0,5,1,59.820345,30.322467


Сохранение файла



In [ ]:
stops.to_file('назем_остановки_СИМ.geojson')

Визуализация

In [ ]:
stops['centroid'] = stops.geometry.centroid

# Добавление столбцов для широты и долготы центроидов
stops['latitude'] = stops['centroid'].y
stops['longitude'] = stops['centroid'].x

# Удаление временного столбца 'centroid', если он не нужен
stops = stops.drop(columns='centroid')

<ipython-input-41-d149c5d51fd8>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  stops['centroid'] = stops.geometry.centroid


In [ ]:
# Создание DataFrame из stops
df = pd.DataFrame(stops)

# Используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')

folium.TileLayer('CartoDB positron').add_to(m)

# Итерация по строкам DataFrame
for idx, row in df.iterrows():
    # Проверяем наличие необходимых колонок и заполняем NaN значения нулями
    parking_spots = row.get('парковочные_места_СИМ', 0)  # Если нет, то 0
    charging_spots = row.get('зарядные_места_СИМ', 0)  # Если нет, то 0

    # Размер точки на основе количества парковочных мест
    parking_size = max(parking_spots / 1, 1)  # Минимальный размер точки 1
    charging_size = max(charging_spots / 1, 1) if charging_spots > 0 else 1  # Минимальный размер точки 1

    # Добавляем круговые маркеры для парковочных мест
    parking_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=parking_size,
            color=None,
            fill=True,
            fill_color='#BFA181',
            fill_opacity=0.6,
            tooltip=f"Парковочные места: {parking_spots}"
        )
    )

    # Добавляем круговые маркеры для зарядных мест
    charging_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=charging_size,
            color=None,
            fill=True,
            fill_color='#0A1828',
            fill_opacity=0.9,
            tooltip=f"Зарядные места: {charging_spots}"
        )
    )

# Добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)

# Добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# Отображаем карту
m

Сохранение карты

In [ ]:
m.save('map_stops.html')

# Станции метро

In [ ]:
city_name_4 = "Московский район, Санкт-Петербург, Россия"

# Загружаем школы (amenity=school)
tags = {'railway': 'station'}
metro = ox.geometries_from_place(city_name_4, tags)

<ipython-input-83-665689a7aa30>:5: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  metro = ox.geometries_from_place(city_name_4, tags)


In [ ]:
metro = gpd.read_file('/content/....geojson')

In [ ]:
gdf = gpd.GeoDataFrame(metro)

# Удаление объектов, не равных "1" в колонке "operator"
gdf_filtered = gdf[gdf['operator'] == 'ГУП «Петербургский метрополитен»']

# Вывод отфильтрованного GeoDataFrame
metro=gdf_filtered

In [ ]:
metro

colour depth            int_name layer  \
element_type osmid                                              
node         249898478   blue    35  Moskovskiye vorota    -5   
             249904416   blue    35          Elekrosila    -5   
             298969888   blue     0            Kupchino   NaN   
             315025552   blue    35         Park Pobedy    -5   
             315051539   blue    22          Zvyozdnaya    -5   
             648222978   blue    29         Moskovskaya    -5   

                                     name             name:en  \
element_type osmid                                              
node         249898478  Московские ворота  Moskovskiye vorota   
             249904416        Электросила          Elekrosila   
             298969888            Купчино            Kupchino   
             315025552        Парк Победы         Park Pobedy   
             315051539           Звёздная          Zvyozdnaya   
             648222978         Московская         Moskovskaya   

                                  name:ru                     network  \
element_type osmid                                                      
node         249898478  Московские ворота  Петербургский метрополитен   
             249904416        Электросила  Петербургский метрополитен   
             298969888            Купчино  Петербургский метрополитен   
             315025552        Парк Победы  Петербургский метрополитен   
             315051539           Звёздная  Петербургский метрополитен   
             648222978         Московская  Петербургский метрополитен   

                                                operator public_transport  \
element_type osmid                                                          
node         249898478  ГУП «Петербургский метрополитен»          station   
             249904416  ГУП «Петербургский метрополитен»          station   
             298969888  ГУП «Петербургский метрополитен»          station   
             315025552  ГУП «Петербургский метрополитен»          station   
             315051539  ГУП «Петербургский метрополитен»          station   
             648222978  ГУП «Петербургский метрополитен»          station   

                        ...                   geometry  \
element_type osmid      ...                              
node         249898478  ...  POINT (30.31807 59.89265)   
             249904416  ...  POINT (30.31907 59.88026)   
             298969888  ...  POINT (30.37549 59.82968)   
             315025552  ...  POINT (30.32149 59.86762)   
             315051539  ...  POINT (30.35146 59.83288)   
             648222978  ...  POINT (30.32238 59.85140)   

                                    network:en                  network:ru  \
element_type osmid                                                           
node         249898478                     NaN                         NaN   
             249904416  Saint Petersburg Metro  Петербургский метрополитен   
             298969888                     NaN                         NaN   
             315025552                     NaN                         NaN   
             315051539                     NaN                         NaN   
             648222978                     NaN                         NaN   

                       network:wikidata esr:user freight operator:branch  \
element_type osmid                                                         
node         249898478              NaN      NaN     NaN             NaN   
             249904416          Q250225      NaN     NaN             NaN   
             298969888              NaN      NaN     NaN             NaN   
             315025552              NaN      NaN     NaN             NaN   
             315051539              NaN      NaN     NaN             NaN   
             648222978              NaN      NaN     NaN             NaN   

                       train uic_ref name:de  
element_type osmid               

In [ ]:
metro.explore()

Метод

In [ ]:
def calculate_metro(geos_layer):
    # Проверяем наличие необходимых атрибутов
    if all(attr in geos_layer.columns for attr in ['парковочные_места_СИМ', 'зарядные_места_СИМ']):

        # Заполняем NaN значения нулями, чтобы избежать ошибок
        geos_layer['парковочные_места_СИМ'] = geos_layer['парковочные_места_СИМ'].fillna(0)
        geos_layer['зарядные_места_СИМ'] = geos_layer['зарядные_места_СИМ'].fillna(0)

        # Проверяем минимальное необходимое количество зарядных мест
        if (geos_layer['парковочные_места_СИМ'] >= 20).all():
            # Если количество парковочных мест достаточно, ничего не делаем
            print("Достаточное количество парковочных мест для СИМ")
        else:
            geos_layer['парковочные_места_СИМ'] = 20

        if (geos_layer['зарядные_места_СИМ'] >= (geos_layer['парковочные_места_СИМ'] * 0.1)).all():
            # Если количество зарядных мест достаточно, ничего не делаем
            print("Достаточное количество зарядных мест для СИМ")
        else:
            geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)

    else:
        # Обработка геослоя
        geos_layer = geos_layer.reset_index(drop=True)
        original_crs = geos_layer.crs
        local_crs = geos_layer.estimate_utm_crs()
        geos_layer = geos_layer.to_crs(local_crs)  # переводим в локальную систему координат

        # Заполняем NaN значения нулями
        geos_layer = geos_layer.fillna(0)

        # Расчёт парковочных и зарядных мест
        geos_layer['парковочные_места_СИМ'] = 20
        geos_layer['зарядные_места_СИМ'] = round(geos_layer['парковочные_места_СИМ'] * 0.1)
        # Теперь удалим все остальные столбцы, кроме 'geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ'
        geos_layer = geos_layer[['geometry', 'name', 'парковочные_места_СИМ', 'зарядные_места_СИМ']]
        geos_layer = geos_layer.to_crs(original_crs)

    return geos_layer

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
metro = calculate_metro(metro)
metro

<ipython-input-92-6c153ac8a516>:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  geos_layer = geos_layer.fillna(0)


,geometry,name,парковочные_места_СИМ,зарядные_места_СИМ
0,POINT (30.31807 59.89265),Московские ворота,20,2.0
1,POINT (30.31907 59.88026),Электросила,20,2.0
2,POINT (30.37549 59.82968),Купчино,20,2.0
3,POINT (30.32149 59.86762),Парк Победы,20,2.0
4,POINT (30.35146 59.83288),Звёздная,20,2.0
5,POINT (30.32238 59.85140),Московская,20,2.0


Сохранение файла

In [ ]:
metro.to_file('станции_метро_СИМ.geojson')

Визуализация

In [ ]:
metro['centroid'] = metro.geometry.centroid

# Добавление столбцов для широты и долготы центроидов
metro['latitude'] = metro['centroid'].y
metro['longitude'] = metro['centroid'].x

# Удаление временного столбца 'centroid', если он не нужен
metro = metro.drop(columns='centroid')

<ipython-input-94-465de5c44a58>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  metro['centroid'] = metro.geometry.centroid


In [ ]:
# Создание DataFrame из metro
df = pd.DataFrame(metro)

# Используем средние значения для центра карты
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Создаем группы для парковочных и зарядных мест
parking_group = folium.FeatureGroup(name='Парковочные места')
charging_group = folium.FeatureGroup(name='Зарядные места')

folium.TileLayer('CartoDB positron').add_to(m)

# Итерация по строкам DataFrame
for idx, row in df.iterrows():
    # Проверяем наличие необходимых колонок и заполняем NaN значения нулями
    parking_spots = row.get('парковочные_места_СИМ', 0)  # Если нет, то 0
    charging_spots = row.get('зарядные_места_СИМ', 0)  # Если нет, то 0

    # Размер точки на основе количества парковочных мест
    parking_size = max(parking_spots / 1, 1)  # Минимальный размер точки 1
    charging_size = max(charging_spots / 1, 1) if charging_spots > 0 else 1  # Минимальный размер точки 1

    # Добавляем круговые маркеры для парковочных мест
    parking_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=parking_size,
            color=None,
            fill=True,
            fill_color='#BFA181',
            fill_opacity=0.6,
            tooltip=f"Парковочные места: {parking_spots}"
        )
    )

    # Добавляем круговые маркеры для зарядных мест
    charging_group.add_child(
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=charging_size,
            color=None,
            fill=True,
            fill_color='#0A1828',
            fill_opacity=0.9,
            tooltip=f"Зарядные места: {charging_spots}"
        )
    )

# Добавляем группы на карту
m.add_child(parking_group)
m.add_child(charging_group)

# Добавляем контрольный слой для переключения между слоями
folium.LayerControl().add_to(m)

# Отображаем карту
m

Сохранение карты

In [ ]:
m.save('map_metro.html')